In [ ]:
# 📦 1. Install OpenCLIP
!pip install git+https://github.com/mlfoundations/open_clip.git

# 🧠 2. Load model and tokenizer from Hugging Face
import open_clip
import torch
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from shutil import copyfile

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Marqo FashionSigLIP from Hugging Face
model_name = 'hf-hub:Marqo/marqo-fashionSigLIP'
model, _, preprocess = open_clip.create_model_and_transforms(model_name)
tokenizer = open_clip.get_tokenizer(model_name)

model.to(device)
model.eval()


In [ ]:
# 🔖 Fashion-related labels
fashion_labels = [
    "white cotton t-shirt for men", "black long-sleeve shirt for men", "striped polo shirt",
    "grey crew neck sweatshirt", "slim-fit dress shirt", "navy blue hoodie for men", "men’s sleeveless tank top",
    "flannel checkered shirt", "men’s linen button-down shirt", "straight-leg blue jeans for men",
    "khaki chino pants", "black cargo pants for men", "tailored wool trousers", "athletic jogger pants",
    "denim shorts for men", "men’s board shorts", "pleated dress pants", "black leather biker jacket for men",
    "men’s trench coat", "padded winter puffer jacket", "bomber jacket in olive green", "grey wool blazer",
    "lightweight windbreaker", "denim jacket for men", "utility field jacket", "compression workout shirt",
    "training shorts for men", "men’s running jacket", "gym stringer tank top", "moisture-wicking t-shirt",
    "sports hoodie for men", "jogger set for men", "men’s yoga pants", "black leather oxford shoes",
    "white sneakers for men", "brown suede loafers", "men’s running shoes", "casual slip-on shoes",
    "black dress boots for men", "formal derby shoes", "leather sandals for men", "leather belt for men",
    "classic wristwatch with metal band", "aviator sunglasses for men", "men’s beanie hat",
    "baseball cap with logo", "crossbody sling bag for men", "men’s briefcase", "leather wallet for men",
    "silk necktie", "patterned pocket square", "relaxed-fit cotton t-shirt for men", "slim-fit navy suit jacket",
    "wool blend overcoat for men", "casual linen pants for men", "ribbed knit sweater for men",
    "tapered black jeans for men", "heavyweight flannel overshirt", "men’s varsity jacket with patches",
    "traditional Vietnamese áo dài for men", "men’s kurta with embroidery", "African print shirt for men",
    "formal tuxedo with bow tie", "business suit in charcoal gray", "men’s wedding sherwani",
    "tailored suit vest for men"
]

# 🔖 Non-fashion-related labels
non_fashion_labels = [
    "mountain landscape", "forest trail", "ocean waves", "houseplant in pot", "wild animals in field",
    "plate of sushi", "latte with foam art", "chocolate cake slice", "glass of wine",
    "smartphone on desk", "computer keyboard", "virtual reality headset", "gaming console controller",
    "modern apartment building", "kitchen interior with cabinets", "historic church facade",
    "city skyline at night", "wooden staircase design",
    "person playing soccer", "child riding a bike", "people dancing at a party", "construction worker in uniform",
    "red sports car", "wooden bookshelf", "toolbox with hammer", "ceramic mug on table"
]


In [ ]:
def is_fashion_image(image_path, threshold=0.8):
    try:
        image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
    except Exception as e:
        print(f"Image error: {e}")
        return False

    # Combine labels and tokenize
    combined_labels = fashion_labels + non_fashion_labels
    text_inputs = tokenizer(combined_labels).to(device)

    with torch.no_grad(), torch.amp.autocast('cuda'):
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_inputs)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

    # Sum probabilities of all fashion-related labels
    fashion_prob = probs[0, :len(fashion_labels)].sum()
    return fashion_prob.item() >= threshold



In [ ]:
from pathlib import Path
from shutil import copyfile
import shutil
from tqdm import tqdm

# 🔧 Setup
input_folder = Path("/kaggle/input/instagram-post-images")
fashion_folder = Path("/kaggle/working/fashion")
non_fashion_folder = Path("/kaggle/working/non_fashion")

fashion_folder.mkdir(parents=True, exist_ok=True)
non_fashion_folder.mkdir(parents=True, exist_ok=True)

# 📂 1. Lọc ảnh thời trang và giữ nguyên cấu trúc thư mục
image_files = list(input_folder.rglob("*.jpg"))
print(f"📸 Found {len(image_files)} images across subfolders")

kept = 0
for img_path in tqdm(image_files, desc="🧵 Filtering fashion images"):
    if is_fashion_image(img_path):  # <-- Giữ nguyên hàm của bạn
        relative_path = img_path.relative_to(input_folder)
        dest_path = fashion_folder / relative_path
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        copyfile(img_path, dest_path)
        kept += 1

print(f"✅ Filtered {kept} fashion images to {fashion_folder}")

# 📦 2. Zip fashion folder
shutil.make_archive(str(fashion_folder), 'zip', str(fashion_folder))
print("✅ Zipped fashion images to /kaggle/working/fashion.zip")

# 📂 3. Lọc ảnh không phải thời trang bằng cách so sánh relative_path
fashion_image_paths = {img.relative_to(fashion_folder) for img in fashion_folder.rglob("*.jpg")}

all_image_files = list(input_folder.rglob("*.jpg"))
non_fashion_count = 0

for img_path in tqdm(all_image_files, desc="🧹 Filtering non-fashion images"):
    relative_path = img_path.relative_to(input_folder)
    if relative_path not in fashion_image_paths:
        dest_path = non_fashion_folder / relative_path
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        copyfile(img_path, dest_path)
        non_fashion_count += 1

print(f"✅ Saved {non_fashion_count} non-fashion images to {non_fashion_folder}")

# 📦 4. Zip non_fashion folder
shutil.make_archive(str(non_fashion_folder), 'zip', str(non_fashion_folder))
print("✅ Zipped non-fashion images to /kaggle/working/non_fashion.zip")
